In [1]:
import sys
import json

In [2]:
# Parse the map from a given filename
def parse_map(filename):
        with open(filename, "r") as f:
                return [[char for char in line] for line in f.read().rstrip("\n").split("\n")]

In [3]:
# Check if a row,col index pair is on the map
def valid_index(pos, n, m):
        return 0 <= pos[0] < n  and 0 <= pos[1] < m

In [4]:
# Find the possible moves from position (row, col)
def moves(map, row, col, visits):
        moves=((row+1,col), (row-1,col), (row,col-1), (row,col+1))

	# Return only moves that are within the board and legal (i.e. go through open space ".")
        return [ move for move in moves if valid_index(move, len(map), len(map[0])) 
                                        and (map[move[0]][move[1]] in ".@" ) 
                                        and (move not in visits)]

In [5]:
def euclid_dist(current_state, target_state):
    return ((current_state[0] - target_state[0])**2 + (current_state[1] - target_state[1])**2)**0.5

In [6]:
def priority_pop(fringe):
    min = fringe[0][3]
    index = 0 
    for i in range(len(fringe)):
        if fringe[i][3] < min:
            min = fringe[i][3]
            index = i
    return fringe.pop(index)


In [11]:
def get_direction(last_move, curr_move):
    if last_move[0] - curr_move[0] == 1:
        return "N"
    elif last_move[0] - curr_move[0] == -1:
        return "S"
    elif last_move[1] - curr_move[1] == 1:
        return "W"
    elif last_move[1] - curr_move[1] == -1:
        return "E"

In [28]:
# Perform search on the map
def search1(house_map):
        # Find pichu start position
        pichu_loc=[(row_i,col_i) for col_i in range(len(house_map[0])) for row_i in range(len(house_map)) if house_map[row_i][col_i]=="p"][0]
        # Find target position (need it to compute heuristic)
        target_loc=[(row_i,col_i) for col_i in range(len(house_map[0])) for row_i in range(len(house_map)) if house_map[row_i][col_i]=="@"][0]
        # Find measure of herutstic to target
        heuristic = euclid_dist(pichu_loc, target_loc)
        
        fringe=[(pichu_loc,0,'',heuristic)]
        visited = []

        while fringe:
            (curr_move, curr_dist, curr_dir, curr_cost)=priority_pop(fringe)
            for move in moves(house_map, *curr_move, visited):
                if house_map[move[0]][move[1]]=="@":
                    return curr_dist+1, curr_dir + get_direction(curr_move, move)
                else:
                    fringe.append((move, curr_dist + 1, curr_dir + get_direction(curr_move, move), euclid_dist(move, target_loc)+curr_dist + 1))
                visited.append(move)
        return 'Inf'

In [33]:
house_map = parse_map('map2.txt')

In [34]:
house_map

[['.', '.', '.', '.', 'X', 'X', 'X'],
 ['.', 'X', 'X', 'X', '.', '.', '.'],
 ['.', '.', '.', '.', 'X', '.', '.'],
 ['.', 'X', '.', 'X', '.', '.', '.'],
 ['.', 'X', '.', 'X', '.', 'X', '.'],
 ['p', 'X', '.', 'X', '@', 'X', '.']]

In [35]:
search1(house_map)

'Inf'